In [2]:
#create CSV output from ESM

from zen_garden.postprocess.results import Results
import pandas as pd
from pathlib import Path

# ---------------- Configuration ----------------
#change according to your setup
dataset_name = "esm_pv_wind"
base_path = Path(r"C:\Users\nicol\GitHub\ZEN-garden\Data\energy_transition_example\outputs")
results_path = base_path / dataset_name
output_dir = Path("./CSV_output") / dataset_name
output_dir.mkdir(parents=True, exist_ok=True)

# ---------------- Load Results ----------------
res_basic = Results(results_path)

capacity = res_basic.get_full_ts("capacity").reset_index()
capacity_addition = res_basic.get_full_ts("capacity_addition").reset_index()
capacity_previous = res_basic.get_full_ts("capacity_previous").reset_index()

# ---------------- Filter Technologies ----------------
selected_technologies = {"heat_pump", "photovoltaics", "wind_offshore", "wind_onshore"}

capacity = capacity[capacity["technology"].isin(selected_technologies)]
capacity_addition = capacity_addition[capacity_addition["technology"].isin(selected_technologies)]
capacity_previous = capacity_previous[capacity_previous["technology"].isin(selected_technologies)]

# ---------------- Adjust capacity_addition["0"] by subtracting capacity_previous["0"] ----------------
# Ensure all year columns are strings
capacity_addition.columns = capacity_addition.columns.map(str)
capacity_previous.columns = capacity_previous.columns.map(str)

# Identify the first year column (usually '0')
year_cols = [col for col in capacity_addition.columns if col.isdigit()]
first_year_col = min(year_cols, key=int)

# Prepare capacity_previous for merging
cap_prev_trimmed = capacity_previous[["technology", "capacity_type", "location", first_year_col]].copy()
cap_prev_trimmed = cap_prev_trimmed.rename(columns={first_year_col: "previous_capacity"})

# Merge and subtract
capacity_addition = pd.merge(
    capacity_addition,
    cap_prev_trimmed,
    on=["technology", "capacity_type", "location"],
    how="left"
)
capacity_addition[first_year_col] = capacity_addition[first_year_col] - capacity_addition["previous_capacity"].fillna(0)
capacity_addition[first_year_col] = capacity_addition[first_year_col].clip(lower=0)
capacity_addition = capacity_addition.drop(columns=["previous_capacity"])

# ---------------- Save Filtered CSVs ----------------
# and round to 4 decimal digits
capacity.round(4).to_csv(output_dir / "capacity.csv", index=False)
capacity_addition.round(4).to_csv(output_dir / "capacity_addition.csv", index=False)
capacity_previous.round(4).to_csv(output_dir / "capacity_previous.csv", index=False)

# ---------------- Aggregation Setup ----------------
specific_locations = {"DE", "CH"}
all_locations = set(capacity["location"].unique())
roe_locations = all_locations - specific_locations


def aggregate_by_location(df, location_set, location_name):
    return (
        df[df["location"].isin(location_set)]
        .groupby(["technology", "capacity_type"])
        .sum(numeric_only=True)
        .assign(location=location_name)
    )


# ---------------- Aggregate Data ----------------
capacity_aggregated = pd.concat([
    aggregate_by_location(capacity, {"DE"}, "DE"),
    aggregate_by_location(capacity, {"CH"}, "CH"),
    aggregate_by_location(capacity, roe_locations, "ROE")
]).reset_index()

capacity_addition_aggregated = pd.concat([
    aggregate_by_location(capacity_addition, {"DE"}, "DE"),
    aggregate_by_location(capacity_addition, {"CH"}, "CH"),
    aggregate_by_location(capacity_addition, roe_locations, "ROE")
]).reset_index()

# ---------------- Save Aggregated CSVs ----------------
capacity_aggregated.round(4).to_csv(output_dir / "capacity_aggregated_by_location.csv", index=False)
capacity_addition_aggregated.round(4).to_csv(output_dir / "capacity_addition_aggregated_by_location.csv", index=False)

print(f"✅ All CSV files for dataset '{dataset_name}' saved successfully in: {output_dir}")

✅ All CSV files for dataset 'energy_transition_ref_2021' saved successfully in: CSV_output\energy_transition_ref_2021


In [3]:
# --- Update aggregated CSV column headers with actual years from system.json ---
import json
# Load system.json
system_path = base_path.parent / dataset_name / "system.json"
with open(system_path, "r") as f:
    system_config = json.load(f)

reference_year = system_config["reference_year"]
interval = system_config["interval_between_years"]
optimized_years = system_config["optimized_years"]
year_labels = [str(reference_year + i * interval) for i in range(optimized_years)]

# Create rename mapping from '0', '1', ... to actual year labels
rename_columns = {str(i): year_labels[i] for i in range(len(year_labels))}

# Function to rename year columns in a CSV
def rename_year_columns(csv_path):
    df = pd.read_csv(csv_path)
    df = df.rename(columns=rename_columns)
    df.to_csv(csv_path, index=False)
    print(f"Renamed year columns in: {csv_path}")

# Apply to both aggregated files
rename_year_columns(output_dir / "capacity_aggregated_by_location.csv")
rename_year_columns(output_dir / "capacity_addition_aggregated_by_location.csv")


Renamed year columns in: CSV_output\energy_transition_ref_2021\capacity_aggregated_by_location.csv
Renamed year columns in: CSV_output\energy_transition_ref_2021\capacity_addition_aggregated_by_location.csv


In [4]:
#specific for PV (write values to demand_yearly_variation.csv)
import pandas as pd
import shutil
from pathlib import Path
import json

# --- CONFIGURATION ---
dataset_name = "energy_transition_ref_2021"

# Base paths
base_path = Path(r"C:\Users\nicol\GitHub\ZEN-garden\Data\energy_transition_example\outputs")
output_dir = Path("./CSV_output") / dataset_name
esm_capacity_path = output_dir / "capacity_addition_aggregated_by_location.csv"

original_dir = Path(r"C:\Users\nicol\GitHub\ZEN-garden\Data_PV\01_SP_MV_PV")
new_dir = Path(r"C:\Users\nicol\GitHub\ZEN-garden\Data_PV\01_SP_MV_PV_new")
original_dyv_path = original_dir / "set_carriers" / "pv_module" / "demand_yearly_variation.csv"
target_dyv_path = new_dir / "set_carriers" / "pv_module" / "demand_yearly_variation.csv"

# --- Step 1: Clone the original directory ---
if new_dir.exists():
    print(f"Directory already exists: {new_dir}")
else:
    shutil.copytree(original_dir, new_dir)
    print(f"Created clone of wind model input: {new_dir}")

# --- Step 2: Load year labels from system.json ---
system_path = base_path.parent / dataset_name / "system.json"
with open(system_path, "r") as f:
    system_config = json.load(f)

reference_year = system_config["reference_year"]
interval = system_config["interval_between_years"]
optimized_years = system_config["optimized_years"]
year_labels = [str(reference_year + i * interval) for i in range(optimized_years)]

# --- Step 3: Load original demand_yearly_variation to preserve all nodes ---
df_existing = pd.read_csv(original_dyv_path)

# --- Step 4: Prepare updated PV data from ESM ---
country_map = {
    "DE": "DEU",
    "CH": "CHE",
    "AT": "AUT",
    "NL": "NLD",
    "ROE": "ROE",
    "ROW": "ROW",
}

df = pd.read_csv(esm_capacity_path)
df_photovoltaics = df[df["technology"] == "photovoltaics"]
df_grouped = df_photovoltaics.groupby(["location"]).sum(numeric_only=True).reset_index()
df_grouped["node"] = df_grouped["location"].map(country_map)
df_grouped = df_grouped[df_grouped["node"].notna()].drop(columns=["location"])

# Transpose to match demand_yearly_variation structure
df_new = df_grouped.set_index("node").T
df_new.index.name = "year"
df_new = df_new.reset_index()

# Fix year column using actual labels from system.json
df_new["year"] = year_labels[:len(df_new)]
df_new["year"] = df_new["year"].astype(str)
df_existing["year"] = df_existing["year"].astype(str)

# Set year as index for merging
df_new = df_new.set_index("year")
df_existing = df_existing.set_index("year")

# --- DEBUG: Check alignment before merge ---
print("✅ Columns available in new data:", df_new.columns.tolist())
print("✅ Columns in existing file:", df_existing.columns.tolist())
print("✅ Years in new data:", df_new.index.tolist())
print("✅ Years in existing file:", df_existing.index.tolist())

# --- Step 5: Merge values only for matching columns ---
updated_cols = [col for col in df_new.columns if col in df_existing.columns]
print("🔄 Columns being updated:", updated_cols)

for col in updated_cols:
    df_existing[col] = df_new[col].combine_first(df_existing[col])

# --- Step 6: Save final file ---
df_result = df_existing.reset_index()
df_result.to_csv(target_dyv_path, index=False)
print(f"✅ Final demand_yearly_variation.csv written to:\n{target_dyv_path}")

Directory already exists: C:\Users\nicol\GitHub\ZEN-garden\Data_PV\01_SP_MV_PV_new
✅ Columns available in new data: ['CHE', 'DEU', 'ROE']
✅ Columns in existing file: ['CHE', 'CHN', 'DEU', 'IND', 'KOR', 'MYS', 'ROA', 'ROE', 'ROW', 'THA', 'USA', 'VNM']
✅ Years in new data: ['2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030']
✅ Years in existing file: ['2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030']
🔄 Columns being updated: ['CHE', 'DEU', 'ROE']
✅ Final demand_yearly_variation.csv written to:
C:\Users\nicol\GitHub\ZEN-garden\Data_PV\01_SP_MV_PV_new\set_carriers\pv_module\demand_yearly_variation.csv


In [5]:
#specific for wind (write values to demand_yearly_variation.csv)
import pandas as pd
import shutil
from pathlib import Path
import json

# --- CONFIGURATION ---
dataset_name = "energy_transition_ref_2021"

# Base paths
base_path = Path(r"C:\Users\nicol\GitHub\ZEN-garden\Data\energy_transition_example\outputs")
output_dir = Path("./CSV_output") / dataset_name
esm_capacity_path = output_dir / "capacity_addition_aggregated_by_location.csv"

original_dir = Path(r"C:\Users\nicol\GitHub\ZEN-garden\Data_Wind\Data_WT")
new_dir = Path(r"C:\Users\nicol\GitHub\ZEN-garden\Data_Wind\Data_WT_new")
original_dyv_path = original_dir / "set_carriers" / "Turbine" / "demand_yearly_variation.csv"
target_dyv_path = new_dir / "set_carriers" / "Turbine" / "demand_yearly_variation.csv"

# --- Step 1: Clone the original directory ---
if new_dir.exists():
    print(f"Directory already exists: {new_dir}")
else:
    shutil.copytree(original_dir, new_dir)
    print(f"Created clone of wind model input: {new_dir}")

# --- Step 2: Load year labels from system.json ---
system_path = base_path.parent / dataset_name / "system.json"
with open(system_path, "r") as f:
    system_config = json.load(f)

reference_year = system_config["reference_year"]
interval = system_config["interval_between_years"]
optimized_years = system_config["optimized_years"]
year_labels = [str(reference_year + i * interval) for i in range(optimized_years)]

# --- Step 3: Load original demand_yearly_variation to preserve all nodes ---
df_existing = pd.read_csv(original_dyv_path)

# --- Step 4: Prepare updated wind data from ESM ---
country_map = {
    "DE": "DEU",
    "CH": "CHE",
    "AT": "AUT",
    "NL": "NLD",
    "ROE": "ROE",
    "ROW": "ROW",
}

df = pd.read_csv(esm_capacity_path)
df_wind = df[df["technology"].isin(["wind_onshore", "wind_offshore"])]
df_grouped = df_wind.groupby(["location"]).sum(numeric_only=True).reset_index()
df_grouped["node"] = df_grouped["location"].map(country_map)
df_grouped = df_grouped[df_grouped["node"].notna()].drop(columns=["location"])

# Transpose to match demand_yearly_variation structure
df_new = df_grouped.set_index("node").T
df_new.index.name = "year"
df_new = df_new.reset_index()

# Fix year column using actual labels from system.json
df_new["year"] = year_labels[:len(df_new)]
df_new["year"] = df_new["year"].astype(str)
df_existing["year"] = df_existing["year"].astype(str)

# Set year as index for merging
df_new = df_new.set_index("year")
df_existing = df_existing.set_index("year")

# --- DEBUG: Check alignment before merge ---
print("✅ Columns available in new data:", df_new.columns.tolist())
print("✅ Columns in existing file:", df_existing.columns.tolist())
print("✅ Years in new data:", df_new.index.tolist())
print("✅ Years in existing file:", df_existing.index.tolist())

# --- Step 5: Merge values only for matching columns ---
updated_cols = [col for col in df_new.columns if col in df_existing.columns]
print("🔄 Columns being updated:", updated_cols)

for col in updated_cols:
    df_existing[col] = df_new[col].combine_first(df_existing[col])

# --- Step 6: Save final file ---
df_result = df_existing.reset_index()
df_result.to_csv(target_dyv_path, index=False)
print(f"✅ Final demand_yearly_variation.csv written to:\n{target_dyv_path}")


Directory already exists: C:\Users\nicol\GitHub\ZEN-garden\Data_Wind\Data_WT_new
✅ Columns available in new data: ['CHE', 'DEU', 'ROE']
✅ Columns in existing file: ['AUS', 'BRA', 'CHE', 'CHN', 'DEU', 'DNK', 'GBR', 'IND', 'JPN', 'NLD', 'ROE', 'ROW', 'SWE', 'USA']
✅ Years in new data: ['2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030']
✅ Years in existing file: ['2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030']
🔄 Columns being updated: ['CHE', 'DEU', 'ROE']
✅ Final demand_yearly_variation.csv written to:
C:\Users\nicol\GitHub\ZEN-garden\Data_Wind\Data_WT_new\set_carriers\Turbine\demand_yearly_variation.csv
